In [ ]:
import datajoint as dj

dj.config["database.host"] = "at-database.ad.bcm.edu"

from neuropixel_pipeline.readers.labview import LabviewNeuropixelMeta
from neuropixel_pipeline.schemata import probe, ephys
from neuropixel_pipeline.helpers import atlab

from pathlib import Path

## Probe

In [ ]:
probe.ProbeType.fill_neuropixel_probes()
atlab.probe_setup() # atlab specific step
probe.ProbeType()

In [ ]:
probe.Probe()

# PreClustering

In [ ]:
# Uses contents only, lookup table
ephys.AcquisitionSoftware()

In [ ]:
# session_meta = {"animal_id": 29187, "session": 18, "scan_idx": 1}
session_meta = {"animal_id": 29187, "session": 8, "scan_idx": 1}

experiment = dj.create_virtual_module("experiment", "pipeline_experiment")

session_rel = experiment.Session() & session_meta
experiment_rig = session_rel.fetch1('rig')

session_meta['rig'] = experiment_rig
ephys.Session.add_session(session_meta, error_on_duplicate=False)
ephys.Session()

In [ ]:
session_path = Path("/mnt/scratch09/Ephys/Mouse/2023-05-16_10-22-55/2023-05-16_10-27-01")
INSERTION_NUMBER = 0

labview_metadata = LabviewNeuropixelMeta.from_h5(session_path)

session_id = (ephys.Session & session_meta).fetch1('session_id')
insertion_key = dict(session_id=session_id, insertion_number=INSERTION_NUMBER)

ephys.ProbeInsertion.insert1(
    dict(
        **insertion_key,
        probe=labview_metadata.serial_number,
    ),
    skip_duplicates=True,
)
ephys.ProbeInsertion()

In [ ]:
# ephys.InsertionLocation.insert(dict())
ephys.InsertionLocation()

In [ ]:
ACQ_SOFTWARE = "LabviewV1"

ephys.EphysFile.insert1(
    dict(
        **insertion_key,
        session_path=session_path,
        acq_software=ACQ_SOFTWARE,
    ),
    skip_duplicates=True,
)
ephys.EphysRecording.populate()
ephys.EphysRecording()

In [ ]:
probe.ElectrodeConfig()

In [ ]:
probe.ElectrodeConfig.Electrode()

In [ ]:
# Mostly fully computed (from neuropixel data I think)
# ephys.LFP.populate()

# Clustering

In [ ]:
# lookup table
ephys.ClusteringMethod()

In [ ]:
settings = {
    "NchanTOT": 385,
    "fs": 30000,
    "nt": 61,
    "Th": 8,
    "spkTh": 8,
    "Th_detect": 9,
    "nwaves": 6,
    "nskip": 25,
    "nblocks": 5,
    "binning_depth": 5,
    "sig_interp": 20,
    "probe_name": "neuropixPhase3B1_kilosortChanMap.mat",
}
settings["nt0min"] = int(20 * settings["nt"] / 61)
settings["NT"] = 2 * settings["fs"]
settings["n_chan_bin"] = settings["NchanTOT"]


# # lookup table, more commonly used (need a simple interface to insert into this occasionally)
ephys.ClusteringParamSet.fill(
    params=settings,
    clustering_method="kilosort4",
    description="default kilosort4 params",
    skip_duplicates=True,
)

ephys.ClusteringParamSet().fetch1()

In [ ]:
# lookup table
ephys.ClusterQualityLabel()

In [ ]:
CLUSTERING_OUTPUT_DIR = session_path / "spikes_kilosort4"

task_source_rel = (ephys.EphysRecording & insertion_key).proj() * (
    ephys.ClusteringParamSet() & {"paramset_idx": 1}
).proj()
task_source_key = task_source_rel.fetch1()

task_source_key["clustering_output_dir"] = CLUSTERING_OUTPUT_DIR
task_source_key["task_mode"] = "load"
ephys.ClusteringTask.insert1(task_source_key, skip_duplicates=True)
ephys.ClusteringTask()

In [ ]:
ephys.Clustering.populate()
ephys.Clustering()

In [ ]:
# lookup table
ephys.CurationType.insert1(dict(curation='maria'), skip_duplicates=True)
ephys.CurationType()

In [ ]:
from datetime import datetime

curation_source_key = (ephys.Clustering() & task_source_key).proj().fetch1()

CURATION_ID = 0
CURATION_OUTPUT_DIR = task_source_key["clustering_output_dir"] # this should be in a new directory due to curation overwriting the original data
CURATION_TIME = datetime.now() # get from phy.log if there is curation
CURATION = "no curation"
CURATION_NOTE = ""

ephys.Curation.insert1(
    dict(
        **curation_source_key,
        curation_id=CURATION_ID,
        curation_time=CURATION_TIME,
        curation_output_dir=CURATION_OUTPUT_DIR,
        curation=CURATION,
        curation_note=CURATION_NOTE,
    ),
    skip_duplicates=True,
)

ephys.Curation()

In [ ]:
ephys.CuratedClustering.populate()
ephys.CuratedClustering()

In [ ]:
ephys.CuratedClustering.Unit()

# PostClustering

In [ ]:
# ephys.WaveformSet.populate()  # requires mean waveforms from ecephys_spike_sorting package (mean_waveforms.npy)
# ephys.WaveformSet()

In [ ]:
# ephys.QualityMetrics.populate()  # requires quality metrics from ecephys_spike_sorting package (metrics.csv)
# ephys.QualityMetrics()